In [ ]:
import pickle

from data_fetching import fetch_solutions_batch, compute_split_solutions
from mechanism import (
    Trade,
    Solution,
    FilterRankRewardMechanism,
    NoFilter,
    BaselineFilter,
    DirectedTokenPairs,
    TokenPairs,
    TradedTokens,
    DirectSelection,
    MonotoneSelection,
    NoReward,
    SubsetFilteringSelection,
    ReferenceReward,
    SingleSurplusSelection,
    AuctionMechanism,
    get_orders,
)

In [ ]:
def compute_all_reward_statistic(solutions_batch, mechanisms, winners_rewards_batch):
    """Compute statistics of winners and rewards"""
    print(f"number of auction: {len(solutions_batch)}")

    # print(f"{len(mechanisms)} mechanisms tested: {mechanisms}")

    winners_batch, rewards_batch = zip(
        *[zip(*winners_rewards) for winners_rewards in winners_rewards_batch])

    print("\naverage number of winners per auction:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(len(winners[k]) for winners in winners_batch) / len(winners_batch)}"
        )

    print("\naverage total score per auction:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(sum(solution.score for solution in winners[k]) for winners in winners_batch) / len(winners_batch) / 10 ** 18}"
        )

    print("\naverage rewards per winner:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(sum(reward for reward, _ in rewards[k].values()) for rewards in rewards_batch) / sum(len(rewards[0]) for rewards in rewards_batch) / 10 ** 18}")

    print("\norder throughput:")
    for k, _ in enumerate(mechanisms):
        settled = sum(len(get_orders(winners[k])) for winners in
                      winners_batch)
        proposed = sum(len(get_orders(solutions)) for solutions in solutions_batch)
        print(
            f"{k}: {settled / proposed}")

    print("\nfrequency of differences between different mechanisms:")
    difference_matrix = {}
    for k in range(len(mechanisms)):
        for l in range(k + 1, len(mechanisms)):
            difference_matrix[(k, l)] = len(list(filter((lambda x: (
                    {solution.id for solution in x[k]} != {solution.id for solution in x[l]})),
                                                        winners_batch))) / len(winners_batch)
    print(difference_matrix)


def compute_reward_statistic(solutions, winners_rewards):
    """Compute statistics of winners and rewards"""
    statistics = []
    for i, (winners, rewards) in enumerate(winners_rewards):
        orders_settled = len(get_orders(winners))
        orders_proposed = len(get_orders(solutions))
        statistic = {
            "orders_settled": orders_settled,
            "orders_proposed": orders_proposed,
        }
        statistics.append(statistic)
    return statistic


def run_analysis(solutions_batch, mechanisms):
    """Run analysis for a batch of solutions

    Runs different mechanisms on the given batch of solutions and computes statistics.
    Parameters
    ----------
    solutions_batch : list[list[Solution]]
        List of solutions for each auction.
    mechanisms : list[AucitonMechanism]
        List of mechanisms to analyze on the given list of solutions.

    Returns
    -------
    all_winners_rewards: list[list[dict[str, tuple[int, int]]]]
        A list of winners and rewards for each auction and each mechanism.
    """
    all_winners_rewards: list[list[dict[str, tuple[int, int]]]] = []
    statistics = []
    for i, solutions in enumerate(solutions_batch):
        winners_rewards = [mechanism.winners_and_rewards(solutions) for mechanism in mechanisms]

        all_winners_rewards.append(winners_rewards)

        statistics.append(compute_reward_statistic(solutions, winners_rewards))

    compute_all_reward_statistic(solutions_batch, mechanisms, all_winners_rewards)

    return statistics


def run_iterative_analysis(solutions_batch, mechanisms):
    total_rewards = []
    for mechanism in mechanisms:
        total_reward_for_mechanism = 0
        for solutions in solutions_batch:
            winners_rewards = settle_iteratively(solutions, mechanism)
            total_reward_for_mechanism += compute_total_reward(winners_rewards)
        total_rewards.append(total_reward_for_mechanism)
    return total_rewards


def settle_iteratively(solutions, mechanism: AuctionMechanism):
    iterative_winners_rewards: list[tuple[list[Solution], dict[str, tuple[int, int]]]] = []
    remaining_solutions = list(solutions)
    while remaining_solutions:
        winners, rewards = mechanism.winners_and_rewards(remaining_solutions)
        iterative_winners_rewards.append((winners, rewards))
        token_pairs = {(trade.sell_token, trade.buy_token) for solution in winners for trade in
                       solution.trades}
        remaining_solutions = [solution for solution in remaining_solutions if all(
            ((trade.sell_token, trade.buy_token) not in token_pairs) for trade in solution.trades)]
    return iterative_winners_rewards


def compute_total_reward(winners_rewards: list[tuple[list[Solution], dict[str, tuple[int, int]]]]):
    total_reward = sum(
        sum(reward for reward, _ in rewards.values()) for _, rewards in winners_rewards)
    return total_reward

Execute one of the following two cell.
1. A set of artificial examples
2. A set of historical auctions

In [ ]:
# handcrafted examples
solutions_batch = [
    [  # batch vs single order solutions
        Solution(
            "batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 3",
            score=150,
            trades=[Trade("2", "C", "D", 150)],
        ),
    ],
    [  # solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=140,
            trades=[Trade("2", "C", "D", 140)],
        ),
        Solution(
            "bad batch",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
    ],
    [  # batch in between solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "batch with overlap",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # reference is not from winner
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on first trade",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # token overlap but not on the same token pair
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=90,
            trades=[Trade("1", "A", "C", 90)],
        ),
    ],
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 100), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="unfair batch",
            solver="solver 2",
            score=110,
            trades=[Trade("1", "A", "B", 50), Trade("2", "A", "C", 60)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 3",
            score=100,
            trades=[Trade("3", "B", "A", 50), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="non-overlapping batch",
            solver="solver 4",
            score=100,
            trades=[Trade("3", "B", "A", 40), Trade("4", "D", "E", 60)],
        ),
        Solution(
            id="non-overlapping batch unfair",
            solver="solver 5",
            score=120,
            trades=[Trade("3", "B", "A", 20), Trade("4", "D", "E", 100)],
        ),
        Solution(
            id="reference A->B",
            solver="solver 1",
            score=80,
            trades=[Trade("1", "A", "B", 80)],
        ),
        Solution(
            id="reference A->C",
            solver="solver 2",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="runner up A->B",
            solver="solver 2",
            score=40,
            trades=[Trade("1", "A", "B", 40)],
        ),
        Solution(
            id="runner up A->C",
            solver="solver 1",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="reference B->A",
            solver="solver 7",
            score=30,
            trades=[Trade("3", "B", "A", 30)],
        ),
        Solution(
            id="reference F->G",
            solver="solver 8",
            score=50,
            trades=[Trade("5", "F", "G", 50)],
        ),
        Solution(
            id="runner up F->G",
            solver="solver 1",
            score=40,
            trades=[Trade("5", "F", "G", 40)],
        ),
        Solution(
            id="reference H->I",
            solver="solver 8",
            score=50,
            trades=[Trade("6", "H", "I", 50)],
        ),
    ],
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

In [ ]:
# fetch auctions from file or database
# this can take around 20 minutes the first time it is run and creates a file of 80MB
auction_start = 10322553 - 50000
auction_end = 10322553
try:
    with open(f"batches_{auction_start}_{auction_end}.pickle", 'rb') as handle:
        solutions_batch = pickle.load(handle)
except FileNotFoundError:
    solutions_batch = fetch_solutions_batch(auction_start, auction_end)
    with open(f"batches_{auction_start}_{auction_end}.pickle", "wb") as handle:
        pickle.dump(solutions_batch, handle, protocol=-1)
solutions_batch_split = [
    compute_split_solutions(solutions, efficiency_loss=0.01, approach="complete") for solutions in
    solutions_batch]

In [ ]:
filtering_function = DirectedTokenPairs()
mechanisms = [
    # our current mechanism
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, same for references
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # same as above but with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, in iteration checking for positive rewards
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # same as above but with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    )]

In [ ]:
# compute results for submitted solutions (Step 1 (+ fairness))
all_results = run_analysis(solutions_batch, mechanisms)

In [ ]:
# compute results for split submitted solutions (Step 2 + 3)
all_results = run_analysis(solutions_batch_split, mechanisms)

More experiments

In [ ]:
total_rewards = run_iterative_analysis(solutions_batch, mechanisms)
[total_reward / 10 ** 18 for total_reward in total_rewards]

In [ ]:
total_rewards = run_iterative_analysis(solutions_batch_split, mechanisms)
[total_reward / 10 ** 18 for total_reward in total_rewards]

In [ ]:
# change for budget (estimate using iterative approach)
total_rewards[-1] / total_rewards[0] - 1

In [ ]:
# run after filtering out solver
solutions_batch_split_filtered = [[solution for solution in solutions if
                                   solution.solver != "0x95480d3f27658e73b2785d30beb0c847d78294c7"]
                                  for solutions in solutions_batch_split]
total_rewards = run_iterative_analysis(solutions_batch_split_filtered, mechanisms)
[total_reward / 10 ** 18 for total_reward in total_rewards]

In [ ]:
# super simple example
solutions_batch = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=250,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 2",
            score=240,
            trades=[Trade("2", "C", "D", 140), Trade("3", "E", "F", 100)],
        ),
    ]
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
]

run_analysis(solutions_batch_split, mechanisms)

In [ ]:
# super simple example
solutions_batch = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=250,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 2",
            score=240,
            trades=[Trade("2", "C", "D", 140), Trade("3", "E", "F", 100)],
        ),
    ]
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
]

run_analysis(solutions_batch_split, mechanisms)
iterative_winners_rewards = settle_iteratively(solutions_batch[0], mechanisms[0])
total_reward = compute_total_reward(iterative_winners_rewards)
total_reward

In [ ]:
# comparison of overlap filtering
mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=DirectedTokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
]

In [ ]:
run_analysis(solutions_batch, mechanisms)

In [ ]:
# run counterfactual analysis
def run_counterfactual_analysis(solutions_batch, mechanisms, remove_executed_orders = True):
    total_rewards = []
    for mechanism in mechanisms:
        total_reward_for_mechanism = 0
        order_uids_settled: set[str] = set()
        for solutions in solutions_batch:
            # filter orders which are already settled
            if remove_executed_orders:
                solutions_filtered = [
                    remove_order_from_solution(solution, order_uids_settled) for
                    solution in solutions
                ]
            else:
                solutions_filtered = list(solutions)
            winners_rewards = mechanism.winners_and_rewards(solutions_filtered)
            total_reward_for_mechanism += compute_total_reward([winners_rewards])
            order_uids_settled.update(get_orders(winners_rewards[0]))
        total_rewards.append(total_reward_for_mechanism)
    return total_rewards


def remove_order_from_solution(solution: Solution, order_uids: set[str]):
    trades_filtered = [trade for trade in solution.trades if trade.id not in order_uids]
    solution_filtered = Solution(
        id=solution.id,
        solver=solution.solver,
        score=sum(trade.score for trade in trades_filtered),
        trades=trades_filtered,
    )
    return solution_filtered


In [ ]:
filtering_function = DirectedTokenPairs()
mechanisms = [
    # our current mechanism
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, in iteration checking for positive rewards
    # with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    )
]
run_counterfactual_analysis(solutions_batch_split, mechanisms)

In [ ]:
run_counterfactual_analysis(solutions_batch_split, mechanisms, False)

In [ ]:
35501 * 0.0022118895837492107

In [ ]:
81550094965831127584 / 64644202650832649410 - 1

In [ ]:
solutions_batch_1 = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="compatible solution",
            solver="solver 2",
            score=100,
            trades=[Trade("3", "E", "F", 100)],
        ),
    ],
    [

        Solution(
            id="compatible solution",
            solver="solver 2",
            score=100,
            trades=[Trade("3", "E", "F", 100)],
        ),
    ]
]

solutions_batch_2 = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="compatible solution",
            solver="solver 2",
            score=100,
            trades=[Trade("3", "E", "F", 100)],
        ),
        Solution(
            id="baseline trade 1",
            solver="solver 2",
            score=100,
            trades=[Trade("1", "A", "B", 90),],
        ),
        Solution(
            id="baseline trade 2",
            solver="solver 2",
            score=100,
            trades=[Trade("2", "C", "D", 90),],
        ),
    ],
    [
        Solution(
            id="compatible solution",
            solver="solver 2",
            score=100,
            trades=[Trade("3", "E", "F", 100)],
        ),
    ]
]

solutions_batch = solutions_batch_2
solutions_batch_split = solutions_batch_split = [
    compute_split_solutions(solutions, efficiency_loss=0.01, approach="complete") for solutions in
    solutions_batch]
filtering_function = DirectedTokenPairs()
mechanisms = [
    # our current mechanism
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, in iteration checking for positive rewards
    # with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    )
]
run_counterfactual_analysis(solutions_batch_split, mechanisms)